In [17]:
from extract_features import extract_features
from config import load_config
import pysam
import pandas as pd
from ml_utils import load_tmap_labels
import os

os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")
print(os.getcwd())

cfg = load_config("project_config/dRNA-ENCFF155CFF_isoquant_config.pkl")
# bam = pysam.AlignmentFile("../" + cfg.bam_file, "rb")  # <-- adjust path if needed

df_tss = pd.read_csv(cfg.tss_labeled_file, dtype={"chrom": str})
df_tes = pd.read_csv(cfg.tes_labeled_file, dtype={"chrom": str})

df_cov = pd.read_csv(cfg.cov_file, sep="\t")
print(f"df_cov.shape: {df_cov.shape}")
df_label = load_tmap_labels(cfg.tmap_file)
print(f"df_label.shape: {df_label.shape}")

# print()

df = df_cov.merge(df_tss, left_on=["tss_chrom", "tss_pos"], right_on=["chrom", "position"], how="inner")
print(f"df.shape: {df.shape}")
df = df.merge(df_tes, left_on=["tes_chrom", "tes_pos"], right_on=["chrom", "position"], how="inner", suffixes=("_tss", "_tes"))
print(f"df.shape: {df.shape}")
df = df.merge(df_label, left_on=["transcript_id"], right_on=["transcript_id"], how="inner")
print(f"df.shape: {df.shape}")


# for col in df.columns:
#     print(f"{col}: {df[col].dtype}")


# distribution of tss/tes labels in positive/negative
df_pos = df[df["label"] == 1]
df_neg = df[df["label"] == 0]

print(f"df_pos.shape: {df_pos.shape}")
print(f"df_neg.shape: {df_neg.shape}")

print(f"df_pos['label_tss   '].value_counts(): {df_pos['label_tss'].value_counts()/df_pos.shape[0]}")
print(f"df_neg['label_tss'].value_counts(): {df_neg['label_tss'].value_counts()/df_neg.shape[0]}")

print(f"df_pos['label_tes'].value_counts(): {df_pos['label_tes'].value_counts()/df_pos.shape[0]}")
print(f"df_neg['label_tes'].value_counts(): {df_neg['label_tes'].value_counts()/df_neg.shape[0]}")










/datadisk1/ixk5174/project_repo/Telos-test
df_cov.shape: (6948, 7)
df_label.shape: (6948, 2)
df.shape: (6948, 88)
df.shape: (6948, 169)
df.shape: (6948, 170)
df_pos.shape: (5534, 170)
df_neg.shape: (1414, 170)
df_pos['label_tss   '].value_counts(): label_tss
1    0.820564
0    0.179436
Name: count, dtype: float64
df_neg['label_tss'].value_counts(): label_tss
1    0.644272
0    0.355728
Name: count, dtype: float64
df_pos['label_tes'].value_counts(): label_tes
1    0.80918
0    0.19082
Name: count, dtype: float64
df_neg['label_tes'].value_counts(): label_tes
1    0.683168
0    0.316832
Name: count, dtype: float64


In [8]:
from ml_utils import load_tmap_labels
from config import load_config
import pandas as pd
import os 
os.chdir("/datadisk1/ixk5174/project_repo/Telos-test/")
print(os.getcwd())

project_config = load_config("project_config/dRNA-ENCFF155CFF_isoquant_config.pkl")
df_cov = pd.read_csv(project_config.cov_file, sep="\t")
df_cov.drop(columns=["strand"], inplace=True)
df_label = load_tmap_labels(project_config.tmap_file)
df_tss = pd.read_csv(project_config.tss_labeled_file, dtype={"chrom": str})
df_tss.drop(columns=["strand"], inplace=True)
df_tes = pd.read_csv(project_config.tes_labeled_file, dtype={"chrom": str})
df_tes.drop(columns=["strand"], inplace=True)

print(f"--------------------------------df_cov.columns--------------------------------")
for x in df_cov.columns:
    print(x)

print(f"--------------------------------merging df_tss--------------------------------")
df = df_cov.merge(df_tss, left_on=["tss_chrom", "tss_pos"], right_on=["chrom", "position"], how="inner")
df.drop(columns=["tss_chrom", "tss_pos", "chrom", "position"], inplace=True)
for x in df.columns:
    print(x)    
print(f"--------------------------------merging df_tes--------------------------------")
df = df.merge(df_tes, left_on=["tes_chrom", "tes_pos"], right_on=["chrom", "position"], how="inner", suffixes=("_tss", "_tes"))
df.drop(columns=["tes_chrom", "tes_pos", "chrom", "position"], inplace=True)
for x in df.columns:
    print(x)
print(f"--------------------------------merging df_label--------------------------------")
df = df.merge(df_label, left_on=["transcript_id"], right_on=["transcript_id"], how="inner")
for x in df.columns:
    print(x)

drop = [
    'chrom_tss','position_tss','chrom_tes','position_tes',
    'tss_chrom','tss_pos','tes_chrom','tes_pos',
    'site_type','site_type_tes','strand', 'strand_tes',
    'ref_id','chrom_num','transcript_id','label', 
    'label_tss', 'label_tes'
]

print(f"--------------------------------drop_features--------------------------------")
drop_features = [c for c in df.columns if c in drop]
for x in drop_features:
    print(x)

print(f"--------------------------------base_features--------------------------------")

base_features = [c for c in df.columns if c not in drop]
for x in base_features:
    print(x)
    


/datadisk1/ixk5174/project_repo/Telos-test
--------------------------------df_cov.columns--------------------------------
transcript_id
tss_chrom
tss_pos
tes_chrom
tes_pos
coverage
--------------------------------merging df_tss--------------------------------
transcript_id
tes_chrom
tes_pos
coverage
total_reads
read_start_density
read_end_density
start_soft_clip_mean
end_soft_clip_mean
start_soft_clip_max
end_soft_clip_max
start_soft_clip_median
end_soft_clip_median
start_soft_clip_count
end_soft_clip_count
mean_mapq
std_mapq
strand_ratio
coverage_before
coverage_after
delta_coverage
nearest_splice_dist
softclip_bias
start_entropy
end_entropy
coverage_gradient_sharpness
max_coverage_gradient
local_avg_coverage
at_content_basic
avg_clip_length
coverage_efficiency
coverage_log_ratio
coverage_ratio
five_prime_degradation_score
gc_content
gc_content_basic
k3_at_rich_kmers
k3_balanced_kmers
k3_gc_kmers_ratio
k3_gc_rich_kmers
k3_kmer_diversity
k3_kmer_entropy
k3_most_frequent_kmer_ratio
k3_p